# Customer Outreach 

In [36]:
import warnings

warnings.filterwarnings("ignore")

In [37]:
from crewai import Agent, Task, Crew

In [38]:
import os
from dotenv import load_dotenv

dotenv_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".env"))

load_dotenv(dotenv_path)

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

In [39]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting–edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
)

In [40]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
)

In [41]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

In [42]:
directory_read_tool = DirectoryReadTool(
    directory="./instructions"
)  # restricted to this directory; how to handle different types of customers
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [43]:
# Let's create a custom tool!
from crewai.tools import BaseTool

In [44]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [45]:
# Download vader lexicon
try:
    nltk.data.find("sentiment/vader_lexicon.zip")
except:
    nltk.download("vader_lexicon")

In [46]:
analyzer = SentimentIntensityAnalyzer()

In [47]:
class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = "Analyzes the sentiment of text to ensure positive and engaging communication."  # when to use this tool and its purpose

    def _run(self, text: str) -> str:
        # manual sentiment analysis tool
        vs = analyzer.polarity_scores(text)

        if vs["compound"] >= 0.05:
            return "positive"
        else:
            return "negative"

In [48]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [49]:
# Create tasks
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector located in {location}"
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[
        directory_read_tool,
        file_read_tool,
        search_tool,
    ],  # read instructions file based off company size, then search info of lead
    agent=sales_rep_agent,
)

In [50]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}. "
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [51]:
crew = Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=True,
    memory=True,
)

In [52]:
inputs = {
    "lead_name": "ethree solutions",
    "industry": "Information Technology Consulting",
    "location": "San Juan, PR",
    "key_decision_maker": "Alberto Cordero",
    "position": "CEO",
    "milestone": "New modern office in beautiful San Juan, Puerto Rico",
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 15c8cd93-50a2-4082-a53b-ad204cc9d72d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Include more detailed financial information and market position in the company background.                   │
│  - Expand on the competitive landscape and how ethree Solutions compares to its competitors.                    │
│  - Add more comprehensive analysis of recent business developments and their implications for potential         │
│  collaboration.                                                                                                 │
│  - Identify specific role needs or challenges faced by key personnel that could align with the services         │
│  offered.                                                                                                       │
│  - Implement a competitor analysis to strengthen the understanding...                                           │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 2044.93ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Task: Conduct an in-depth analysis of ethree solutions, a company in the Information Technology Consulting     │
│  sector located in San Juan, PRthat recently showed interest in our solutions. Utilize all available data       │
│  sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and      │
│  potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy      │
│  effectively.                                                                                                   │
│  Don't make assumptions and only use information you absolutely sure about.                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Subject:** Unlock Your Vision for a Sustainable Future                                                       │
│                                                                                                                 │
│  Dear Cordero,                                                                                                  │
│                                                                                                                 │
│  Congratulations on the opening of your new modern office in beautiful San Juan! This exciting milestone is     │
│  not just a testament to ethree solutions’ growth, but a reflection of your commitment to leading the charge    │
│  in digital transformation and sustainability.                                                                  │
│                                                                                                                 │
│  At CrewAI, we share your passion for innovation and operational excellence. Your recent projects showcasing    │
│  significant cost savings and contributions to decarbonization resonate deeply with our mission to empower      │
│  businesses to thrive sustainably.                                                                              │
│                                                                                                                 │
│  As you settle into your new space, imagine enhancing your operational capabilities with our process            │
│  optimization solutions. Our tools can help streamline your workflows, making it easier for your team to focus  │
│  on what truly matters – driving impactful change for your clients.                                             │
│                                                                                                                 │
│  Moreover, our adaptive technologies, including advanced analytics and AI tools, can elevate your               │
│  decision-making processes, leading to more strategic interventions in energy efficiency and sustainability     │
│  initiatives.                                                                                                   │
│                                                                                                                 │
│  We understand that fostering technological adoption is crucial for your team. To that end, we would be         │
│  thrilled to discuss how our tailored training programs can upskill your staff, aligning perfectly with your    │
│  consulting services and corporate identity.                                                                    │
│                                                                                                                 │
│  Let’s explore a partnership that will not only enhance the value you deliver to your clients but also          │
│  position ethree solutions as a beacon of innovation in the industry.                                           │
│                                                                                                                 │
│  Looking forward to connecting soon!                                                                            │
│                                                                                                                 │
│  Best regards,                                         

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: afc4add9-ca82-4f31-aa83-9920deead8e7                                                                     │
│  Agent: Lead Sales Representative                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 15c8cd93-50a2-4082-a53b-ad204cc9d72d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Subject:** Unlock Your Vision for a Sustainable Future                                         │
│                                                                                                                 │
│  Dear Cordero,                                                                                                  │
│                                                                                                                 │
│  Congratulations on the opening of your new modern office in beautiful San Juan! This exciting milestone is     │
│  not just a testament to ethree solutions’ growth, but a reflection of your commitment to leading the charge    │
│  in digital transformation and sustainability.                                                                  │
│                                                                                                                 │
│  At CrewAI, we share your passion for innovation and operational excellence. Your recent projects showcasing    │
│  significant cost savings and contributions to decarbonization resonate deeply with our mission to empower      │
│  businesses to thrive sustainably.                                                                              │
│                                                                                                                 │
│  As you settle into your new space, imagine enhancing your operational capabilities with our process            │
│  optimization solutions. Our tools can help streamline your workflows, making it easier for your team to focus  │
│  on what truly matters – driving impactful change for your clients.                                             │
│                                                                                                                 │
│  Moreover, our adaptive technologies, including advanced analytics and AI tools, can elevate your               │
│  decision-making processes, leading to more strategic interventions in energy efficiency and sustainability     │
│  initiatives.                                                                                                   │
│                                                                                                                 │
│  We understand that fostering technological adoption is crucial for your team. To that end, we would be         │
│  thrilled to discuss how our tailored training programs can upskill your staff, aligning perfectly with your    │
│  consulting services and corporate identity.                                                                    │
│                                                                                                                 │
│  Let’s explore a partnership that will not only enhance the value you deliver to your clients but also          │
│  position ethree solutions as a beacon of innovation in the industry.                                           │
│                                                                                                                 │
│  Looking forward to connecting soon!                                                                            │
│                                                       

In [53]:
from IPython.display import Markdown

Markdown(result.raw)

**Subject:** Unlock Your Vision for a Sustainable Future

Dear Cordero,

Congratulations on the opening of your new modern office in beautiful San Juan! This exciting milestone is not just a testament to ethree solutions’ growth, but a reflection of your commitment to leading the charge in digital transformation and sustainability.

At CrewAI, we share your passion for innovation and operational excellence. Your recent projects showcasing significant cost savings and contributions to decarbonization resonate deeply with our mission to empower businesses to thrive sustainably.

As you settle into your new space, imagine enhancing your operational capabilities with our process optimization solutions. Our tools can help streamline your workflows, making it easier for your team to focus on what truly matters – driving impactful change for your clients.

Moreover, our adaptive technologies, including advanced analytics and AI tools, can elevate your decision-making processes, leading to more strategic interventions in energy efficiency and sustainability initiatives.

We understand that fostering technological adoption is crucial for your team. To that end, we would be thrilled to discuss how our tailored training programs can upskill your staff, aligning perfectly with your consulting services and corporate identity.

Let’s explore a partnership that will not only enhance the value you deliver to your clients but also position ethree solutions as a beacon of innovation in the industry.

Looking forward to connecting soon!

Best regards,

[Your Name]  
Lead Sales Representative  
CrewAI